In [2]:
import pandas as pd
import numpy as np
# import openpyxl

In [23]:
df = pd.read_csv('../data/all_data.csv')

In [29]:
df.head()

,film_name,IMDb,director,actors,image,description
0,Человек-паук: Нет пути домой,8.2,Джон Уоттс,"Джон Уоттс, Том Холланд, Зендая, Бенедикт Камб...",https://resizer.mail.ru/p/77287c77-f712-5110-8...,"Теперь, когда весь мир узнал, кто скрывается з..."
1,Джентльмены,7.8,Гай Ричи,"Гай Ричи, Мэттью Макконахи, Чарли Ханнэм, Хью ...",https://resizer.mail.ru/p/269f49ca-a456-59bf-a...,"Талантливый выпускник Оксфорда, применив свой ..."
2,Выживший,8.0,Алехандро Гонсалес Иньярриту,"Алехандро Гонсалес Иньярриту, Леонардо ДиКапри...",https://resizer.mail.ru/p/e8c2b865-d24d-585b-b...,Легендарный первопроходец Хью Гласс серьезно р...
3,Гнев человеческий,7.1,Гай Ричи,"Гай Ричи, Джейсон Стэйтем, Алекс Фернс, Холт М...",https://resizer.mail.ru/p/61fc89c3-5ccc-598a-9...,Загадочный и холодный на вид Эйч (Джейсон Стэй...
4,1+1,8.5,Оливье Накаш,"Оливье Накаш, Эрик Толедано, Франсуа Клюзе, Ом...",https://resizer.mail.ru/p/25e85ca5-28f1-5cd6-8...,В результате несчастного случая богатый бизнес...


In [3]:
df = pd.read_excel('../data/Hand_parsed.xlsx', engine='openpyxl')

### Использование rubert-tiny2. Простая работа с одним bi-encoder
https://huggingface.co/cointegrated/rubert-tiny2

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('cointegrated/rubert-tiny2')
sentences = ["привет мир", "hello world", "здравствуй вселенная"]
embeddings = model.encode(sentences)
print(embeddings) # каждый документ представляется вектором длиной 312

/Users/alex_eyfler/ds_bootcamp/2d-phase-projects/find_my_movie/.env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


[[ 1.93479415e-02 -2.03980207e-02 -8.45527742e-03 -6.85829073e-02
  -1.67199329e-03  3.86166871e-02 -1.27368402e-02 -3.14126462e-02
  -7.30952919e-02 -4.57527526e-02  1.95689127e-02  7.20972717e-02
   8.39341134e-02  4.11882214e-02  3.29586007e-02 -6.46812022e-02
   1.07747212e-01  1.74314752e-02 -4.82148863e-03  4.41244766e-02
   4.88842726e-02  2.57910900e-02  4.95995507e-02 -7.43958401e-03
   6.54861182e-02  1.71001684e-02 -7.56175146e-02  7.08268676e-03
  -4.15337794e-02  5.64554483e-02 -4.71331142e-02 -5.52772656e-02
   1.06138386e-01 -2.34868992e-02 -1.73601545e-02  1.06622368e-01
  -3.67118679e-02 -3.43236551e-02 -1.06261641e-01  1.56027339e-02
  -9.50848535e-02  1.52671203e-01  1.28692567e-01 -4.05506119e-02
   9.49789956e-03  1.58098564e-02  5.39585389e-02 -2.82900166e-02
   2.27003973e-02 -5.47641590e-02  4.87862006e-02 -4.12375592e-02
   7.44173257e-03 -4.16501313e-02 -4.70889136e-02  5.22873364e-02
   1.19798936e-01  2.35627517e-02 -5.26659600e-02 -5.11540240e-03
   3.04333

In [4]:
# получаем в список все свои описания
descriptions = df['description'].to_list()
# получаем список эмбедингов
# descriptions_embedded = model.encode(descriptions)

In [28]:
user_text = 'фильм про спасение планеты супергероями'
user_text_embedded = model.encode(user_text)

In [ ]:
import torch
from sentence_transformers.util import cos_sim # косинусное сходство, посмотреть еще какие есть

cos_scores = cos_sim(user_text_embedded, descriptions_embedded)[0]
# print(cos_scores[0])
# Получаем индексы топ-7 самых похожих фильмов
top_results = torch.topk(cos_scores, k=7)
print(top_results)

# Выводим результаты
print("\nТоп-7 фильмов, соответствующих вашему запросу:\n")
for score, idx in zip(top_results[0], top_results[1]):
    idx = idx.item()
    print(f"Название фильма: {df.iloc[idx]['movie_title']}")
    print(f"Режиссер: {df.iloc[idx]['director']}")
    print(f"В главных ролях: {df.iloc[idx]['main_starring']}")
    print(f"Описание: {df.iloc[idx]['description']}")
    print(f"Сходство: {score.item():.3f}")
    print("-" * 50)

### Пробуем комбинированный подход bi-encoder + faiss и затем cross-encoder

In [ ]:
# Импорт необходимых библиотек
import faiss
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Шаг 1: Загрузка данных
df = pd.read_excel('../data/Hand_parsed.xlsx', engine='openpyxl')
descriptions = df['description'].tolist()

# Шаг 2: Вычисление эмбеддингов описаний фильмов с помощью SentenceTransformer
# Используем модель для получения эмбеддингов
bi_encoder_model = SentenceTransformer('cointegrated/rubert-tiny2')
description_embeddings = bi_encoder_model.encode(descriptions, convert_to_numpy=True)
# print(type(description_embeddings))
# print(description_embeddings.shape)
# print(description_embeddings)

# Преобразуем эмбеддинги в формат float32 для использования в FAISS
description_embeddings = np.array(description_embeddings, dtype='float32')

# Шаг 3: Создание индекса FAISS
dimension = description_embeddings.shape[1]  # Размерность эмбеддингов
index = faiss.IndexFlatL2(dimension)         # Используем L2 расстояние
index.add(description_embeddings)            # Добавляем эмбеддинги в индекс
# print(type(index))
# print(index)

# Шаг 4: Обработка пользовательского запроса
# user_query = input("Введите описание фильма или запрос: ")
user_query = 'фильм про спасение планеты супергероями'

# Вычисляем эмбеддинг пользовательского запроса
query_embedding = bi_encoder_model.encode([user_query], convert_to_numpy=True)
query_embedding = np.array(query_embedding, dtype='float32')

# Шаг 5: Поиск топ-N кандидатов с помощью FAISS
N = 100  # Число кандидатов для переранжирования !!!!!! надо будет поставить 50
distances, indices = index.search(query_embedding, N)
# print(type(distances), distances.shape)
# print(distances)
# print(type(indices), indices.shape)
# print(indices)

# Извлекаем описания кандидатов
candidate_indices = indices[0] # список индексов кандидатов
candidate_descriptions = [descriptions[idx] for idx in candidate_indices]
# print(type(candidate_descriptions))
# print(candidate_descriptions)
# print(len(candidate_descriptions))

# Шаг 6: Переранжирование кандидатов с помощью Cross-Encoder
# Загрузка модели Cross-Encoder
## ---------------------------------------------------------------------------
## Этот шаг локально не загрузился и не сработал, пришлось разбираться с API
cross_encoder_model_name = 'cointegrated/rubert-base-cased-nli-threeway'
tokenizer = AutoTokenizer.from_pretrained(cross_encoder_model_name)
cross_encoder_model = AutoModelForSequenceClassification.from_pretrained(cross_encoder_model_name)

# Функция для переранжирования
def rerank_with_cross_encoder(query, texts):
    scores = []
    for text in tqdm(texts, desc="Переранжирование кандидатов"):
        # Подготовка входных данных для модели
        inputs = tokenizer.encode_plus(query, text, return_tensors='pt', truncation=True, max_length=512)
        # Получение логитов (предсказания)
        with torch.no_grad():
            logits = cross_encoder_model(**inputs).logits
        # Получение вероятности класса "entailment" (соответствие)
        probs = torch.softmax(logits, dim=1)
        entailment_prob = probs[0][2].item()  # Индекс 2 соответствует классу "entailment"
        scores.append(entailment_prob)
    return scores

# Вычисляем новые оценки сходства с помощью Cross-Encoder
rerank_scores = rerank_with_cross_encoder(user_query, candidate_descriptions)

# Соединяем индексы кандидатов и их оценки
results = list(zip(candidate_indices, rerank_scores))

# Сортируем результаты по оценкам в порядке убывания
results.sort(key=lambda x: x[1], reverse=True)
## ---------------------------------------------------------------------------

# Шаг 7: Вывод топ-K результатов
K = 3  # Число результатов для вывода
top_results = results[:K]

print("\nТоп-{} фильмов, соответствующих вашему запросу:\n".format(K))
for idx, score in top_results:
    print(f"Название фильма: {df.iloc[idx]['movie_title']}")
    print(f"Режиссер: {df.iloc[idx]['director']}")
    print(f"В главных ролях: {df.iloc[idx]['main_starring']}")
    print(f"Описание: {df.iloc[idx]['description']}")
    print(f"Сходство: {score:.3f}")
    print("-" * 50)

In [32]:
# будем получать эмбеддинги локально, а переранжировать с помощью API
# рабочий код, полученный в процессе отладки (с принтами)
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import json
import faiss
import torch
from huggingface_hub import InferenceClient
from tqdm import tqdm
import requests
from time import time


#№ 1. Загрузка данных
df = pd.read_csv('../data/all_data.csv')
descriptions = df['description'].tolist()

## 2. Определение модели для локального использования
bi_encoder_model = SentenceTransformer('cointegrated/rubert-tiny2')

## 3. Получение эмбеддингов для описаний фильмов из датасета
description_embeddings = bi_encoder_model.encode(descriptions, convert_to_numpy=True)
# print('Step 3!')
# print(type(description_embeddings))
# print(description_embeddings.shape)
# print(description_embeddings)

# Преобразуем эмбеддинги в формат float32 для использования в FAISS
description_embeddings = np.array(description_embeddings, dtype='float32')

## 4. Создание индекса FAISS
dimension_of_ind = description_embeddings.shape[1]  # Размерность эмбеддингов
index_faiss = faiss.IndexFlatL2(dimension_of_ind)   # Используем L2 расстояние
index_faiss.add(description_embeddings)             # Добавляем эмбеддинги в индекс


## 5. Обработка пользовательского запроса
# user_query = input("Введите описание фильма или запрос: ")
user_query = 'фильм про спасение планеты супергероями'

# Вычисляем эмбеддинг пользовательского запроса
query_embedding = bi_encoder_model.encode([user_query], convert_to_numpy=True)
query_embedding = np.array(query_embedding, dtype='float32')
# print('Step 5!')
# print(type(query_embedding))
# print(query_embedding.shape)
# print(query_embedding)

## 6. Поиск топ-N кандидатов с помощью FAISS
N = 50  # Число кандидатов для переранжирования !!!!!! надо будет поставить 50
distances, indices = index_faiss.search(query_embedding, N)
# print('Step 6!')
# print(type(distances), distances.shape)
# print(distances)
# print(type(indices), indices.shape)
# print(indices)

# Извлекаем описания кандидатов
candidate_indices = indices[0] # список индексов кандидатов
candidate_descriptions = [descriptions[idx] for idx in candidate_indices]
# print('Candidates:')
# print(f'type candidate_indices : {type(candidate_indices)}')

## ---------------------------------------------------------------------------
## Выше все работает
## ---------------------------------------------------------------------------

# Теперь получим переранжирование с помощью cross-encoder через API

cross_encoder_model_name = 'cointegrated/rubert-base-cased-nli-threeway'
API_TOKEN = '#####'
NLI_API_URL = f"https://api-inference.huggingface.co/models/{cross_encoder_model_name}"
HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}

def query_nli(premise, hypothesis):
    """
    Отправляет запрос к API модели NLI и возвращает вероятность 'entailment'.
    """
    inputs = f"{premise} [SEP] {hypothesis}"
    payload = {
        "inputs": inputs,
        "parameters": {
            "candidate_labels": ["entailment", "contradiction", "neutral"]
        }
    }
    response = requests.post(NLI_API_URL, headers=HEADERS, json=payload)
    if response.status_code != 200:
        raise ValueError(f"Request failed with status code {response.status_code}: {response.text}")
    output = response.json()
    # Проверяем наличие необходимых ключей в ответе
    if 'labels' in output and 'scores' in output:
        labels = output['labels']
        scores = output['scores']
        proba = dict(zip(labels, scores))
        entailment_score = proba.get('entailment', 0.0)
        return entailment_score
    else:
        raise ValueError(f"Unexpected response format: {output}")

def get_entailment_scores(user_query, descriptions):
    """
    Для каждого описания из списка candidate_descriptions вычисляет вероятность 'entailment' с user_query.
    Возвращает список вероятностей.
    """
    entailment_scores = []
    for description in tqdm(descriptions, desc='Переранжирование результатов'):
        try:
            score = query_nli(user_query, description)
            entailment_scores.append(score)
            # Можно добавить задержку между запросами, чтобы избежать ограничения по частоте запросов
            # time.sleep(0.5)  # Например, полсекунды
        except ValueError as e:
            print(f"Ошибка при обработке описания: {e}")
            entailment_scores.append(0.0)  # По умолчанию добавим 0 в случае ошибки
    return entailment_scores

rerank_scores = get_entailment_scores(user_query, candidate_descriptions)

# Соединяем индексы кандидатов и оценки после переранжирования
results = list(zip(candidate_indices, rerank_scores))

# Сортируем итоговые результаты по rerank_scores в порядке убывания
results.sort(key=lambda x: x[1], reverse=True)

# Вывод результатов числа K результатов
K = 10
top_results = results[:K]

print(f"Топ-{K} фильмов, соответствующих вашему запросу:")
for idx, score in top_results:
    print(f"Название фильма: {df.iloc[idx]['film_name']}")
    print(f"Режиссер: {df.iloc[idx]['director']}")
    print(f"В главных ролях: {df.iloc[idx]['actors']}")
    print(f"Описание: {df.iloc[idx]['description']}")
    print(f"Сходство: {score:.3f}")
    print("-" * 50)

Переранжирование результатов: 100%|██████████| 50/50 [00:38<00:00,  1.28it/s]

Топ-10 фильмов, соответствующих вашему запросу:
Название фильма: Пираты Галактики Барракуда
Режиссер: Антон Борматов
В главных ролях: Антон Борматов, Борис Дейков, Полина Гухман, Алексей Онежен, Юрий Колокольников, Софья Лебедева, Лукерья Ильяшенко, Ольга Науменко, Роман Курцын, Татьяна Рассказова, Россия, Альберт Рябышев, Александр Плотников, Юрий Сапронов, Андрей Геласимов, Лотос Суни Парк, Валерий Царьков
Описание: Космический пират из далекой-далекой галактики Барракуда, обладающий суперспособностями, оказывается на планете Земля. Скрываясь от наказания, которое грозит ему по суровым космическим законам, пришелец принимает облик тихого отличника Мити. Его цель — найти ретранслятор, который когда-то был оставлен на Земле, и отправить сигнал бедствия своей команде. Случайным образом пират спасает девочку Кристину от травли одноклассников и примыкает к компании юного хулигана Коли. Теперь им всем нужно ускользнуть от преследования инопланетных змеелюдей и… обезвредить бомбу, способную

### Сохранение данных для использования в стримлите

In [33]:
bi_encoder_model.save('../data/sasha/local_bi_encoder')

In [34]:
# Сохранение эмбеддингов на диск
np.save('../data/description_embeddings.npy', description_embeddings)

### Ниже простые тесты, чтобы понять, как работает API

In [10]:
API_URL = "https://api-inference.huggingface.co/models/cointegrated/rubert-base-cased-nli-threeway"
headers = {"Authorization": "Bearer #######"}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

text1 = user_query
text2 = candidate_descriptions[0]

inputs = f"{text1} [SEP] {text2}"

output = query({
    "inputs": inputs,
    "parameters": {
            "candidate_labels": ["entailment", "neutral", "contradiction"]
        }
})

print(output)

{'sequence': 'фильм про спасение планеты супергероями [SEP] Сюжет фильма «Мстители: Финал»: Мстители и Стражи Галактики вступают в последнюю стадию войны с Таносом, владеющим всемогущей перчаткой бесконечности. Грядет финальная битва между силами героев и Безумным Титаном, которая раз и навсегда определит дальнейшую судьбу не только Земли, но и всей Вселенной.', 'labels': ['entailment', 'contradiction', 'neutral'], 'scores': [0.5770885944366455, 0.3668346405029297, 0.056076813489198685]}


In [12]:
print(type(output))
print(output)

<class 'dict'>
{'sequence': 'фильм про спасение планеты супергероями [SEP] Сюжет фильма «Мстители: Финал»: Мстители и Стражи Галактики вступают в последнюю стадию войны с Таносом, владеющим всемогущей перчаткой бесконечности. Грядет финальная битва между силами героев и Безумным Титаном, которая раз и навсегда определит дальнейшую судьбу не только Земли, но и всей Вселенной.', 'labels': ['entailment', 'contradiction', 'neutral'], 'scores': [0.5770885944366455, 0.3668346405029297, 0.056076813489198685]}


In [13]:
# Извлекаем список меток и вероятностей
labels = output['labels']
scores = output['scores']

# Находим индекс класса 'entailment'
entailment_index = labels.index('entailment')

# Получаем соответствующую вероятность
entailment_score = scores[entailment_index]

print(f"Вероятность для 'entailment': {entailment_score}")

Вероятность для 'entailment': 0.5770885944366455
